# Hamiltonian Exponentiation - 3rd Place Janczar Knurek, Poland

Below is code and methodology from the third place solution to the Spring 2022 Classiq Coding Competition. 



## Submission Description
First I sort the hamiltonian by absolute value of coefficients. I remove some of them (the smallest). To simulate I use a variation of the second-order trotter formula, but i do not split the hamiltonian in half, rather i split only the largest summands in half. I tried to reimplement ZZZZ gates by choosing the head of the fountain at random, but it achieved worse results than choosing always the highest qubit. I came up with an idea that choosing the same fountain head in neighbouring gates allows more cancellation, so i implemented it and it saved circa 60 depth.

In [1]:
from qiskit.opflow import *

from qiskit.quantum_info import Pauli

def _read_ham(filename):
    with open(filename) as input_file:
        lines = [x for x in input_file.readlines() if x != "" and x != "\n"]
        lines[0] = "+ " + lines[0]
    for line in lines:
        sgn, coeff, _, pauli = line.split()
        coeff = float(coeff) if sgn == "+" else -float(coeff)
        op = PauliOp(Pauli(pauli), coeff)
        yield op


In [2]:
def dist(A, B):
    return np.linalg.norm(A - B, ord=2)

In [3]:
raw_hamiltonian = list(_read_ham("input.txt"))

target_U = sum(raw_hamiltonian).exp_i().to_matrix()

sorted_hamiltonian = sorted(raw_hamiltonian, key=lambda x: abs(x.coeff))

trimmed_hamiltonian = sum(sorted_hamiltonian[80:-1])

In [4]:
import qiskit
from qiskit import QuantumCircuit, transpile
import qiskit.synthesis
from qiskit.synthesis import SuzukiTrotter
from qiskit.circuit.library import PauliEvolutionGate

from qiskit.quantum_info import Pauli
from random import choice, shuffle

from numpy import trace

from qiskit import execute
from qiskit import Aer

import numpy as np

from numpy.random import permutation

def _make_random_fountain(pauli: Pauli, w, previous=None):
    
    qc = QuantumCircuit(len(pauli))
    nontrivials = [i for i, p in enumerate(pauli) if p != Pauli("I")]
    if not nontrivials:
        return qc
    if previous is not None and previous in nontrivials:
        target_qubit = previous
    else:
        target_qubit = nontrivials[0]
    diag = QuantumCircuit(len(pauli))
    for i in nontrivials:
        if pauli[i] == Pauli('X'):
            diag.sdg(i)
        if pauli[i] in [Pauli('Y'), Pauli('X')]:
            diag.h(i)
    qc = qc.compose(diag)
    for i in nontrivials:
        if i != target_qubit:
            qc.cx(i, target_qubit)
    qc.rz(w * 2, target_qubit)
    for i in nontrivials:
        if i != target_qubit:
            qc.cx(i, target_qubit)
    qc = qc.compose(diag.inverse())
    return qc, target_qubit

class FountainMaker:
    
    def __init__(self):
        self.history = [None]
    
    def __call__(self, pauli: Pauli, w):
        result, target_qubit =  _make_random_fountain(pauli, w, self.history[-1])
        self.history.append(target_qubit)
        return result

print(_make_random_fountain(Pauli("XZY"), 0.5))

# tmp_h = list(Hx)
# shuffle(tmp_h)
# print(tmp_h)

# Hxx = sum(tmp_h)

def su_dist(U, Ul):
    λ = trace(Ul.conj().T @ U) / 1024
    return dist(λ * Ul, U)

def other_su_dist(U, Ul):
    return dist(U / U[0,0], Ul / Ul[0, 0])

def run_experiments(H, times=10):
    results = []
    
    for i in range(times):
        suzuki = SuzukiTrotter(order=1, reps=1, atomic_evolution=FountainMaker())
#         suzuki = SuzukiTrotter(order=1, reps=1, cx_structure='fountain')
        H1t = PauliEvolutionGate(H[130:] / 2, 1.0)
        H2t = PauliEvolutionGate(H[:130], 1)
        res1 = suzuki.synthesize(H1t)
        res2 = suzuki.synthesize(H2t)
        
        qc1 = res1
        qc1 = qc1.compose(res2)
        qc1 = qc1.compose(res1)
        circ = transpile(qc1, optimization_level=3, basis_gates=['u3', 'cx'])
        if circ.depth() < min(results, key=lambda x: x[0], default=(10000, 0))[0]:
            print("BEST! ", end='')
        results.append((circ.depth(), circ))
        print(f"{len(circ)=}, {circ.depth()=}", end='')
        res = execute(circ, backend=Aer.get_backend('unitary_simulator'))
        Ul = np.asarray(res.result().get_unitary())
        print(f" distance={su_dist(target_U, Ul)}")
        tmp_h = list(H)
        tmp_h = H[40:] + H[:40]
        H = sum(tmp_h)
    return results
        
results = run_experiments(trimmed_hamiltonian[30:], 1)
        

(<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f60efc631f0>, 0)


<ipython-input-4-fb5e9e2181bb>:75: DeprecationWarning: SuzukiTrotter for odd orders is deprecated as of 0.20.0, and will be removed no earlier than 3 months after that release date. Suzuki product formulae are symmetric and therefore only defined for evenorders.
  suzuki = SuzukiTrotter(order=1, reps=1, atomic_evolution=FountainMaker())


BEST! len(circ)=1341, circ.depth()=775 distance=0.09834909498911042


In [6]:
good_circuit = results[0][1]
print(good_circuit.qasm(), file=open("result.qasm", 'w'))